In [ ]:
# filter_human_hybrid_transparent.py

import os
import cv2
import numpy as np


INPUT_DIR     = r"C:\Users\golds\Desktop\Data-Driven Design\jewelry_images\jewelry_filter"
OUT_KEEP      = os.path.join("output", r"C:\Users\golds\Desktop\Data-Driven Design\jewelry_images\jewelry_filter_keep")
OUT_DISCARD   = os.path.join("output", r"C:\Users\golds\Desktop\Data-Driven Design\jewelry_images\jewelry_filter_discard")


FACE_CASCADE     = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"


SKIN_LOWER       = np.array([0,  48, 80], dtype=np.uint8)
SKIN_UPPER       = np.array([20, 255,255], dtype=np.uint8)
SKIN_BLOB_THRESH = 0.02   # 2%，


BG_GRAY_THRESH   = 10


os.makedirs(OUT_KEEP, exist_ok=True)
os.makedirs(OUT_DISCARD, exist_ok=True)


face_detector = cv2.CascadeClassifier(FACE_CASCADE)

def detect_face(img_gray):
    faces = face_detector.detectMultiScale(
        img_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))
    return len(faces) > 0

def largest_skin_blob_ratio(bgr):
    hsv    = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    mask   = cv2.inRange(hsv, SKIN_LOWER, SKIN_UPPER)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    mask   = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel, iterations=2)
    mask   = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num_labels <= 1:
        return 0.0
    areas   = stats[1:, cv2.CC_STAT_AREA]
    max_area= areas.max()
    img_area= bgr.shape[0] * bgr.shape[1]
    return max_area / img_area

def make_transparent(bgr):

    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    alpha = np.where(gray > BG_GRAY_THRESH, 255, 0).astype(np.uint8)
    b, g, r = cv2.split(bgr)
    return cv2.merge([b, g, r, alpha])

def process_image(path):
    img = cv2.imread(path)
    if img is None:
        print(f"[WARN] error：{path}")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fn   = os.path.splitext(os.path.basename(path))[0] + ".png"

    # 1) face
    if detect_face(gray):
        out_dir, tag = OUT_DISCARD, "FACE"
    else:
        # 2) skin color
        ratio = largest_skin_blob_ratio(img)
        if ratio > SKIN_BLOB_THRESH:
            out_dir, tag = OUT_DISCARD, f"SKIN({ratio:.2%})"
        else:
            out_dir, tag = OUT_KEEP,    "OK"

    # 3)  PNG
    rgba = make_transparent(img)
    dst_path = os.path.join(out_dir, fn)
    cv2.imwrite(dst_path, rgba)
    print(f"[{tag:>10}] {fn} → {('keep' if out_dir==OUT_KEEP else 'discard')}")

if __name__ == "__main__":
    exts = (".png",".jpg",".jpeg",".bmp",".tiff")
    files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(exts)]
    if not files:
        print(f"[ERROR] `{INPUT_DIR}` no image files found")
        exit(1)

    for f in files:
        process_image(os.path.join(INPUT_DIR, f))

    print("✅ fliter done")
    print(f"  keep → {len(os.listdir(OUT_KEEP))}  (PNG, transparent)")
    print(f"  discard → {len(os.listdir(OUT_DISCARD))} 张 (PNG, transparent)")


[SKIN(2.53%)] 0000.png → 丢弃
[SKIN(4.50%)] 0001.png → 丢弃
[        OK] 0002.png → 保留
[      FACE] 0003.png → 丢弃
[        OK] 0004.png → 保留
[SKIN(22.89%)] 0005.png → 丢弃
[SKIN(4.26%)] 0006.png → 丢弃
[SKIN(6.61%)] 0007.png → 丢弃
[      FACE] 0008.png → 丢弃
[        OK] 0009.png → 保留
[SKIN(9.57%)] 0010.png → 丢弃
[SKIN(7.45%)] 0011.png → 丢弃
[        OK] 0012.png → 保留
[SKIN(17.73%)] 0013.png → 丢弃
[SKIN(9.59%)] 0014.png → 丢弃
[      FACE] 0015.png → 丢弃
[SKIN(3.57%)] 0016.png → 丢弃
[        OK] 0017.png → 保留
[        OK] 0018.png → 保留
[        OK] 0019.png → 保留
[        OK] 0020.png → 保留
[SKIN(6.08%)] 0021.png → 丢弃
[        OK] 0022.png → 保留
[SKIN(22.22%)] 0023.png → 丢弃
[        OK] 0024.png → 保留
[SKIN(2.51%)] 0025.png → 丢弃
[        OK] 0026.png → 保留
[SKIN(54.35%)] 0027.png → 丢弃
[        OK] 0028.png → 保留
[        OK] 0029.png → 保留
[      FACE] 0030.png → 丢弃
[      FACE] 0031.png → 丢弃
[        OK] 0032.png → 保留
[SKIN(3.19%)] 0033.png → 丢弃
[        OK] 0034.png → 保留
[SKIN(6.80%)] 0035.png → 丢弃
[      F